<a href="https://colab.research.google.com/github/Yuji-Sakata0110/my-second-blogsite/blob/master/TensorFlow_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
import tensorflow as tf 

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

11501568/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/12
340/469 [====================>.........] - ETA: 43s - loss: 2.2927 - accuracy: 0.1217

KeyboardInterrupt: ignored

In [2]:
import tensorflow as tf

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.14768305, -0.59544736, -0.59300494, -0.07125088,  0.33009863,
        -0.3031472 , -0.5362161 ,  0.11586209,  0.38465047, -0.04341025]],
      dtype=float32)

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.12255239, 0.0582887 , 0.05843125, 0.0984555 , 0.14707671,
        0.07807801, 0.06184552, 0.11871406, 0.1553229 , 0.10123509]],
      dtype=float32)

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.550047

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [10]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 9s 4ms/step - loss: 0.3088 - accuracy: 0.9112
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1539 - accuracy: 0.9550
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1131 - accuracy: 0.9654
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0921 - accuracy: 0.9712
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0773 - accuracy: 0.9760


In [11]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0790 - accuracy: 0.9755 - 970ms/epoch - 3ms/step


[0.07904838770627975, 0.9754999876022339]

In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.2280186e-07, 6.9771975e-09, 2.4960456e-05, 4.2927370e-04,
        6.1605651e-12, 6.0505516e-07, 4.0791432e-13, 9.9954444e-01,
        1.5426455e-07, 2.8309321e-07],
       [9.1645971e-09, 1.1945231e-04, 9.9983156e-01, 5.4126822e-06,
        4.2662187e-13, 3.5875531e-05, 4.5316392e-06, 2.6589455e-11,
        3.1270533e-06, 3.2491943e-11],
       [2.1738444e-07, 9.9850011e-01, 7.6177188e-05, 1.9316135e-06,
        5.4206073e-05, 7.0218048e-06, 4.4783178e-06, 1.2909074e-03,
        6.4543245e-05, 3.9744293e-07],
       [9.9990320e-01, 3.3897307e-09, 3.1946436e-06, 4.4829552e-08,
        3.9272418e-08, 2.1028973e-06, 1.6649261e-05, 7.3192154e-05,
        1.9899244e-09, 1.6428564e-06],
       [8.0943886e-05, 8.6596632e-09, 7.5256918e-05, 8.9906770e-08,
        9.5632058e-01, 5.1654188e-06, 2.5934311e-05, 2.2750798e-04,
        5.2437929e-07, 4.3263901e-02]], dtype=float32)>